# Part 1 Get Weather Dscription and Precipitation

### Generate Random Coordinates

In [1]:
# Import the dependencies.
import pandas as pd
import numpy as np
import sys
# Import the datetime module from the datetime library.
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
SIZE = 1500

lats = np.random.uniform(low=-90.000, high=90.000, size=SIZE)
lngs = np.random.uniform(low=-180.000, high=180.000, size=SIZE)
lat_lngs = zip(lats, lngs)

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

### Get the nearby cities

In [3]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Create a list of cities near given coordinates.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # Only take unique city.
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count.
num = len(cities)
print(f"Found {num} cities near our coordinates.") 

Found 618 cities near our coordinates.


### Open Weather Map request setup

In [4]:
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key

# Starting URL for Weather Map API Call.
URL = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [5]:
# example
city_url = URL + "&q=" + "San Francisco"
city_weather = requests.get(city_url).json()
city_weather

{'coord': {'lon': -122.42, 'lat': 37.77},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 56.64,
  'feels_like': 50.25,
  'temp_min': 50,
  'temp_max': 62.6,
  'pressure': 1015,
  'humidity': 44},
 'visibility': 16093,
 'wind': {'speed': 5.82, 'deg': 310},
 'clouds': {'all': 75},
 'dt': 1582353110,
 'sys': {'type': 1,
  'id': 5817,
  'country': 'US',
  'sunrise': 1582296758,
  'sunset': 1582336462},
 'timezone': -28800,
 'id': 5391959,
 'name': 'San Francisco',
 'cod': 200}

### function to return rain / snow fall  
return rain / snow in last 3 hours, unit inches.  
if KeyError, set to zero; other except, raise exception again.

In [6]:
# return rain_snow_amount in inch.
def city_rain_snow_inch(city_weather, rain_snow):
    try:
        rain_snow_inch = city_weather[rain_snow]["3h"] / 25.4
        return rain_snow_inch
    except KeyError:
        return 0.0
    except:
        e = sys.exc_info()[0]
        print(f"{rain_snow} parsing: {e}. Skipping...")
        raise e

### reqest Open Weather Map

In [7]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("-----------------------------")
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    if (i >= 10):
        break

    # Create endpoint URL with each city.
    city_url = URL + "&q=" + city

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()

        # Parse out the needed data.
        city_name = city_weather["name"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_min_temp = city_weather["main"]["temp_min"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        # use "weather", "description"
        city_wx_dscpt = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        # convert mm to inch.
        city_rain = city_rain_snow_inch(city_weather, "rain")
        city_snow = city_rain_snow_inch(city_weather, "snow")
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        # Append the city information into city_data list.
        city_data.append({"City": city_name,
                          "Lat":  city_lat,
                          "Lng":  city_lng,
                          "Min Temp": city_min_temp,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Weather": city_wx_dscpt, 
                          "Country": city_country,
                          "Rain": city_rain,
                          "Snow": city_snow,
                          "Date": city_date})

    except KeyError:
        print(f"{city} not found. Skipping...")
        pass        
    # If an error is experienced, skip the city.
    except:
        e = sys.exc_info()[0]
        print(f"{city} weather parsing: {e}. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
taolanaro not found. Skipping...
kuche not found. Skipping...
amderma not found. Skipping...
belushya guba not found. Skipping...
illoqqortoormiut not found. Skipping...
temaraia not found. Skipping...
juifang not found. Skipping...
mullaitivu not found. Skipping...
toftir not found. Skipping...
barentsburg not found. Skipping...
dzhusaly not found. Skipping...
samusu not found. Skipping...
vaitupu not found. Skipping...
bolungarvik not found. Skipping...
naftah not found. Skipping...
jabiru not found. Skipping...
panixtlahuaca not found. Skipping...
tarudant not found. Skipping...
bur gabo not found. Skipping...
lolua not found. Skipping...
nizhneyansk not found. Skipping...
saint-pascal not found. Skipping...
samalaeulu not found. Skipping...
ondorhaan not found. Skipping...
marcona not found. Skipping...
kemijarvi not found. Skipping...
mrirt not found. Skipping...
mys shmidta not found. Skipping...
taburi not found. Skippi

### create DataFrame

In [8]:
city_data_df = pd.DataFrame(city_data)
new_column_order = ["City", "Country", "Date", "Lat", "Lng",
                    "Weather", "Min Temp", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Rain", "Snow"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(2)

,City,Country,Date,Lat,Lng,Weather,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Rain,Snow
0,Parana,AR,2020-02-22 06:33:45,-31.73,-60.52,clear sky,62.01,63.00,68,0,4.0,0.0,0.000000
1,Krasnosel'kup,RU,2020-02-22 06:33:45,65.70,82.47,light snow,6.21,6.21,94,99,13.4,0.0,0.005118


### save DataFrame in csv file

In [9]:
city_data_df.to_csv('weather_data\WeatherPy_challenge.csv', index_label="City_ID")

### total cities

In [10]:
total_cities = len(city_data_df.index)
print(f"There are {total_cities} cities in {output_file}.")

There are 561 cities in weather_data\WeatherPy_challenge.csv.
